# Requirement Text File Run

In [28]:
!pip install -r requirements.txt

  Using cached PennyLane-0.35.1-py3-none-any.whl.metadata (9.1 kB)
  Using cached tensorflow-2.17.1-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached scikit_learn-1.4.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached medmnist-3.0.1-py3-none-any.whl.metadata (13 kB)
  Using cached torch-2.2.2-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Using cached autograd-1.6.2-py3-none-any.whl.metadata (706 bytes)
  Using cached numpy-2.2.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tensorflow_intel-2.17.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached scikit_image-0.25.2-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached fire-0.7.0.tar.gz (87 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached ml_dtypes-0.4.1-cp311-cp311-win_amd64.whl.metad

ERROR: Cannot install -r requirements.txt (line 1), -r requirements.txt (line 3), -r requirements.txt (line 4), -r requirements.txt (line 6), numpy==2.2.1 and tensorflow because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts



# Data
---



In [29]:
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
pca32 = ['pca32-1', 'pca32-2', 'pca32-3', 'pca32-4']
autoencoder32 = ['autoencoder32-1', 'autoencoder32-2', 'autoencoder32-3', 'autoencoder32-4']
pca30 = ['pca30-1', 'pca30-2', 'pca30-3', 'pca30-4']
autoencoder30 = ['autoencoder30-1', 'autoencoder30-2', 'autoencoder30-3', 'autoencoder30-4']
pca16 = ['pca16-1', 'pca16-2', 'pca16-3', 'pca16-4', 'pca16-compact']
autoencoder16 = ['autoencoder16-1', 'autoencoder16-2', 'autoencoder16-3', 'autoencoder16-4', 'autoencoder16-compact']
pca12 = ['pca12-1', 'pca12-2', 'pca12-3', 'pca12-4']
autoencoder12 = ['autoencoder12-1', 'autoencoder12-2', 'autoencoder12-3', 'autoencoder12-4']

def data_load_and_process(dataset, classes=[0, 1], feature_reduction='resize256', binary=True):
    if dataset == 'fashion_mnist':
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

    elif dataset == 'mnist':
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    #3d


    x_train, x_test = x_train[..., np.newaxis] / 255.0, x_test[..., np.newaxis] / 255.0

    print(f"Loaded X_train shape: {x_train.shape}, Y_train shape: {y_train.shape}")
    print(f"Loaded X_test shape: {x_test.shape}, Y_test shape: {y_test.shape}")
    if classes == 'odd_even':
        odd = [1, 3, 5, 7, 9]
        X_train = x_train #4d
        X_test = x_test
        if binary == False:
            Y_train = [1 if y in odd else 0 for y in y_train]
            Y_test = [1 if y in odd else 0 for y in y_test]
        elif binary == True:
            Y_train = [1 if y in odd else -1 for y in y_train]
            Y_test = [1 if y in odd else -1 for y in y_test]

    elif classes == '>4':
        greater = [5, 6, 7, 8, 9]
        X_train = x_train
        X_test = x_test
        if binary == False:
            Y_train = [1 if y in greater else 0 for y in y_train]
            Y_test = [1 if y in greater else 0 for y in y_test]
        elif binary == True:
            Y_train = [1 if y in greater else -1 for y in y_train]
            Y_test = [1 if y in greater else -1 for y in y_test]

    else:
        x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
        x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
        '''After resizing256 with fashion_mnist, The shape (12000, 256) indicates that after filtering, you only have 12,000 samples left in X_train. This is likely due to the filtering step where you only keep samples that belong to the specified classes.'''
        '''for mnist data after resizing, we have (12665,256) data due to filtering'''

        X_train, X_test = x_train[x_train_filter_01], x_test[x_test_filter_01]
        Y_train, Y_test = y_train[x_train_filter_01], y_test[x_test_filter_01]

        if binary == False:
            Y_train = [1 if y == classes[0] else 0 for y in Y_train]
            Y_test = [1 if y == classes[0] else 0 for y in Y_test]
        elif binary == True:
            Y_train = [1 if y == classes[0] else -1 for y in Y_train]
            Y_test = [1 if y == classes[0] else -1 for y in Y_test]

    if feature_reduction == 'resize256':
        X_train = tf.image.resize(X_train[:], (256, 1)).numpy()
        X_test = tf.image.resize(X_test[:], (256, 1)).numpy()
        X_train, X_test = tf.squeeze(X_train).numpy(), tf.squeeze(X_test).numpy()
        return X_train, X_test, Y_train, Y_test

    elif feature_reduction == 'pca8' or feature_reduction in pca32 \
            or feature_reduction in pca30 or feature_reduction in pca16 or feature_reduction in pca12:

        X_train = tf.image.resize(X_train[:], (784, 1)).numpy()
        X_test = tf.image.resize(X_test[:], (784, 1)).numpy()
        X_train, X_test = tf.squeeze(X_train), tf.squeeze(X_test)

        if feature_reduction == 'pca8':
            pca = PCA(8)
        elif feature_reduction in pca32:
            pca = PCA(32)
        elif feature_reduction in pca30:
            pca = PCA(30)
        elif feature_reduction in pca16:
            pca = PCA(16)
        elif feature_reduction in pca12:
            pca = PCA(12)


        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)

        # Rescale for angle embedding
        if feature_reduction == 'pca8' or feature_reduction == 'pca16-compact' or \
                feature_reduction in pca30 or feature_reduction in pca12:
            X_train, X_test = (X_train - X_train.min()) * (np.pi / (X_train.max() - X_train.min())),\
                              (X_test - X_test.min()) * (np.pi / (X_test.max() - X_test.min()))
        return X_train, X_test, Y_train, Y_test

    elif feature_reduction == 'autoencoder8' or feature_reduction in autoencoder32 \
            or feature_reduction in autoencoder30 or feature_reduction in autoencoder16 or feature_reduction in autoencoder12:
        if feature_reduction == 'autoencoder8':
            latent_dim = 8
        elif feature_reduction in autoencoder32:
            latent_dim = 32
        elif feature_reduction in autoencoder30:
            latent_dim = 30
        elif feature_reduction in autoencoder16:
            latent_dim = 16
        elif feature_reduction in autoencoder12:
            latent_dim = 12



        class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim
                self.encoder = tf.keras.Sequential([
                    layers.Flatten(),
                    layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                    layers.Dense(784, activation='sigmoid'),
                    layers.Reshape((28, 28))
                ])

            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded

        autoencoder = Autoencoder(latent_dim)

        autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        autoencoder.fit(X_train, X_train,
                        epochs=10,
                        shuffle=True,
                        validation_data=(X_test, X_test))

        X_train, X_test = autoencoder.encoder(X_train).numpy(), autoencoder.encoder(X_test).numpy()

        # Rescale for Angle Embedding
        # Note this is not a rigorous rescaling method
        if feature_reduction == 'autoencoder8' or feature_reduction == 'autoencoder16-compact' or\
                feature_reduction in autoencoder30 or feature_reduction in autoencoder12:
            X_train, X_test = (X_train - X_train.min()) * (np.pi / (X_train.max() - X_train.min())), \
                              (X_test - X_test.min()) * (np.pi / (X_test.max() - X_test.min()))

        return X_train, X_test, Y_train, Y_test


#Unitary
---


In [30]:
# This module contains the set of unitary ansatze that will be used to benchmark the performances of Quantum Convolutional Neural Network (QCNN) in QCNN.ipynb module
import pennylane as qml

# Unitary Ansatze for Convolutional Layer
def U_TTN(params, wires):  # 2 params cc_1
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])


def U_5(params, wires):  # 10 params CC_7
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])
    qml.RZ(params[2], wires=wires[0])
    qml.RZ(params[3], wires=wires[1])
    qml.CRZ(params[4], wires=[wires[1], wires[0]])
    qml.CRZ(params[5], wires=[wires[0], wires[1]])
    qml.RX(params[6], wires=wires[0])
    qml.RX(params[7], wires=wires[1])
    qml.RZ(params[8], wires=wires[0])
    qml.RZ(params[9], wires=wires[1])


def U_6(params, wires):  # 10 params CC_8
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])
    qml.RZ(params[2], wires=wires[0])
    qml.RZ(params[3], wires=wires[1])
    qml.CRX(params[4], wires=[wires[1], wires[0]])
    qml.CRX(params[5], wires=[wires[0], wires[1]])
    qml.RX(params[6], wires=wires[0])
    qml.RX(params[7], wires=wires[1])
    qml.RZ(params[8], wires=wires[0])
    qml.RZ(params[9], wires=wires[1])


def U_9(params, wires):  # 2 params cc_2
    qml.Hadamard(wires=wires[0])
    qml.Hadamard(wires=wires[1])
    qml.CZ(wires=[wires[0], wires[1]])
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])


def U_13(params, wires):  # 6 params cc_4
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CRZ(params[2], wires=[wires[1], wires[0]])
    qml.RY(params[3], wires=wires[0])
    qml.RY(params[4], wires=wires[1])
    qml.CRZ(params[5], wires=[wires[0], wires[1]])


def U_14(params, wires):  # 6 params cc_5
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CRX(params[2], wires=[wires[1], wires[0]])
    qml.RY(params[3], wires=wires[0])
    qml.RY(params[4], wires=wires[1])
    qml.CRX(params[5], wires=[wires[0], wires[1]])


def U_15(params, wires):  # 4 params cc_3
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[2], wires=wires[0])
    qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])


def U_SO4(params, wires):  # 6 params CC_6
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[2], wires=wires[0])
    qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[4], wires=wires[0])
    qml.RY(params[5], wires=wires[1])


def U_SU4(params, wires): # 15 params CC_9
    qml.U3(params[0], params[1], params[2], wires=wires[0])
    qml.U3(params[3], params[4], params[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[6], wires=wires[0])
    qml.RZ(params[7], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[8], wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.U3(params[9], params[10], params[11], wires=wires[0])
    qml.U3(params[12], params[13], params[14], wires=wires[1])




# Pooling Layer



def custom_Pooling1(params, wires): #10 params
    if len(params) < 10:  # Adjust based on your needs
        raise ValueError("Params must have at least 10 elements")
    
    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply controlled-Y gates
    qml.CY(wires=[wires[0], wires[1]]) 
    qml.CY(wires=[wires[0], wires[2]]) 
    qml.CY(wires=[wires[0], wires[3]]) 

    # Apply controlled-Y gates
    qml.CY(wires=[wires[1], wires[2]]) 
    qml.CY(wires=[wires[1], wires[3]]) 
    qml.CY(wires=[wires[2], wires[3]])

#Can't use it as it takes 5 qubits
def custom_Pooling2(params, wires): #14 parameters, 5 wires
    if len(params) < 14:  # Adjust based on your needs
        raise ValueError("Params must have at least 14 elements")

    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    qml.Hadamard(wires=wires[4])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply RX gates to the second half of qubits
    qml.RX(params[10], wires=wires[0])
    qml.RX(params[11], wires=wires[1])
    qml.RX(params[12], wires=wires[2])
    qml.RX(params[13], wires=wires[3])

    
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.CNOT(wires=[wires[1], wires[2]])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.CNOT(wires=[wires[3], wires[0]])

    qml.CY(wires=[wires[0], wires[4]])
    qml.CY(wires=[wires[1], wires[4]])
    qml.CY(wires=[wires[2], wires[4]])
    qml.CY(wires=[wires[3], wires[4]])
 
    qml.Hadamard(wires=wires[4])




def mod_circuit_a(params, wires): #2 params

    qml.CRZ(params[0], wires=[wires[0], wires[1]]) 
    qml.X(wires=wires[0])
    qml.CRX(params[1], wires=[wires[0], wires[1]])

def mod_circuit_b(params, wires): #4 paarams
    qml.RY(params[0], wires=wires[0])  # RY rotation on wire 0
    qml.RY(params[1], wires=wires[1])  # RY rotation on wire 1
    qml.CNOT(wires=[wires[0], wires[1]])  # CNOT with control wire 0 and target wire 1
    qml.CRZ(params[2], wires=[wires[0], wires[1]])  # Controlled RZ gate
    qml.X(wires=wires[0])  # Pauli-X gate on wire 0
    qml.CRX(params[3], wires=[wires[0], wires[1]])  # Controlled RX gate


def mod_circuit_c(params, wires): #4params
    qml.Hadamard(wires=wires[0])  # Hadamard gate on wire 0
    qml.Hadamard(wires=wires[1])  # Hadamard gate on wire 1
    qml.CZ(wires=[wires[0], wires[1]])  # Controlled-Z gate
    qml.RX(params[0], wires=wires[0])  # RX rotation on wire 0
    qml.RX(params[1], wires=wires[1])  # RX rotation on wire 1
    qml.CRZ(params[2], wires=[wires[0], wires[1]])  # Controlled RZ gate
    qml.X(wires=wires[0])  # Pauli-X gate on wire 0
    qml.CRX(params[3], wires=[wires[0], wires[1]])  # Controlled RX gate

def custom_Pooling3_Mod_b(params, wires): #22 parameters, 4 wires
    if len(params) < 22:  # Adjust based on your needs
        raise ValueError("Params must have at least 22 elements")
    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits

    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])


    # Apply MOD gates (using mod_circuit_b)
    mod_circuit_b([params[10], params[11], params[12], params[13]], wires=[wires[0], wires[1]])  # MOD gate for qubits 0 and 1
    mod_circuit_b([params[14], params[15], params[16], params[17]], wires=[wires[2], wires[3]])  # MOD gate for qubits 2 and 3
    mod_circuit_b([params[18], params[19], params[20], params[21]], wires=[wires[1], wires[3]])  # MOD gate for qubits 1 and 3 



def custom_ansatz3_Mod_c(params, wires): # 22 parameters, 4 wires
    if len(params) < 22:  # Adjust based on your needs
        raise ValueError("Params must have at least 22 elements")
    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply MOD gates (using mod_circuit_c)
    mod_circuit_c([params[10], params[11], params[12], params[13]], wires=[wires[0], wires[1]])  # MOD gate for qubits 0 and 1
    mod_circuit_c([params[14], params[15], params[16], params[17]], wires=[wires[2], wires[3]])  # MOD gate for qubits 2 and 3
    mod_circuit_c([params[18], params[19], params[20], params[21]], wires=[wires[1], wires[3]])  # MOD gate for qubits 1 and 3


def custom_ansatz3_c_LessParam(params, wires): # 12 parameters, 4 wires
    if len(params) < 12:  # Adjust based on your needs
        raise ValueError("Params must have at least 12 elements")
    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply MOD gates (using mod_circuit_c)
    mod_circuit_c([params[10], params[11], params[4], params[5]], wires=[wires[0], wires[1]])  # MOD gate for qubits 0 and 1
    mod_circuit_c([params[6], params[7], params[8], params[9]], wires=[wires[2], wires[3]])  # MOD gate for qubits 2 and 3
    mod_circuit_c([params[10], params[11], params[6], params[7]], wires=[wires[1], wires[3]])  # MOD gate for qubits 1 and 3

def custom_Pooling3_Mod_a(params, wires): # 16 parameters, 4 wires
    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply MOD gates (using mod_circuit_a)
    mod_circuit_a([params[10], params[11]], wires=[wires[0], wires[1]])  # MOD gate for qubits 0 and 1
    mod_circuit_a([params[12], params[13]], wires=[wires[2], wires[3]])  # MOD gate for qubits 2 and 3
    mod_circuit_a([params[14], params[15]], wires=[wires[1], wires[3]])  # MOD gate for qubits 1 and 3


def custom_Pooling4(params, wires): #14 parameters
    if len(params) < 14:  # Adjust based on your needs
        raise ValueError("Params must have at least 14 elements")

    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply RX gates to the second half of qubits
    qml.RX(params[10], wires=wires[0])
    qml.RX(params[11], wires=wires[1])
    qml.RX(params[12], wires=wires[2])
    qml.RX(params[13], wires=wires[3])

    
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.CNOT(wires=[wires[1], wires[2]])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.CNOT(wires=[wires[3], wires[0]])






def Pooling_ansatz1(params, wires): #2 params
    qml.CRZ(params[0], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRX(params[1], wires=[wires[0], wires[1]])

def Pooling_ansatz2(wires): #0 params
    qml.CRZ(wires=[wires[0], wires[1]])

def Pooling_ansatz3(*params, wires): #3 params
    qml.CRot(*params, wires=[wires[0], wires[1]])


# Angular Hybrid

---

In [31]:
# This is an implementation of an alternative Mottonen State Preparation to avoid normalization problem.
import pennylane as qml

# 3 bits of information is embedded in 2 wires
def Angular_Hybrid_2(X, wires):
    qml.RY(X[0], wires=wires[0])

    qml.PauliX(wires=wires[0])
    qml.CRY(X[1], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRY(X[2], wires=[wires[0], wires[1]])

# 15 bits of information is embedded in 4 wires
def Angular_Hybrid_4(X, wires):
    qml.RY(X[0], wires=wires[0])

    qml.PauliX(wires=wires[0])
    qml.CRY(X[1], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRY(X[2], wires=[wires[0], wires[1]])

    qml.RY(X[3], wires=wires[2])
    qml.CNOT(wires=[wires[1], wires[2]])
    qml.RY(X[4], wires=wires[2])
    qml.CNOT(wires=[wires[0], wires[2]])
    qml.RY(X[5], wires=wires[2])
    qml.CNOT(wires=[wires[1], wires[2]])
    qml.RY(X[6], wires=wires[2])
    qml.CNOT(wires=[wires[0], wires[2]])

    qml.RY(X[7], wires=wires[3])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.RY(X[8], wires=wires[3])
    qml.CNOT(wires=[wires[1], wires[3]])
    qml.RY(X[9], wires=wires[3])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.RY(X[10], wires=wires[3])
    qml.CNOT(wires=[wires[0], wires[3]])
    qml.RY(X[11], wires=wires[3])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.RY(X[12], wires=wires[3])
    qml.CNOT(wires=[wires[1], wires[3]])
    qml.RY(X[13], wires=wires[3])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.RY(X[14], wires=wires[3])
    qml.CNOT(wires=[wires[0], wires[3]])

# Embedding

---

In [ ]:
import pennylane as qml
from pennylane import numpy as np

def data_embedding(X, embedding_type='Amplitude'):
    """Encodes the input classical data into a quantum state."""
    n_qubits = 8  # Assuming 8 qubits based on your device initialization

    if embedding_type == 'Amplitude':
        # Amplitude Embedding
        num_features = len(X)
        if num_features > 2**n_qubits:
            raise ValueError(f"Input size ({num_features}) exceeds the capacity of {n_qubits} qubits for amplitude embedding.")
        norm = np.linalg.norm(X)
        if norm == 0:
            raise ValueError("Input vector has zero norm.")
        normalized_X = X / norm
        padding = [0.0] * (2**n_qubits - num_features)
        amplitude_data = np.concatenate((normalized_X, padding))
        qml.AmplitudeEmbedding(amplitude_data, wires=range(n_qubits), normalize=False)

    elif embedding_type == 'Angle':
        # Angle Embedding
        for i in range(min(len(X), n_qubits)):
            qml.RX(X[i], wires=i)

    elif embedding_type == 'Angle-compact':
        # Compact Angle Embedding (assuming pairs of features per qubit)
        num_pairs = len(X) // 2
        if num_pairs > n_qubits:
            raise ValueError(f"Input size ({len(X)}) requires more than {n_qubits} qubits for compact angle embedding.")
        for i in range(num_pairs):
            qml.RY(X[2 * i], wires=i)
            qml.RZ(X[2 * i + 1], wires=i)

    elif embedding_type.startswith('Amplitude-Hybrid4-'):
        # Hybrid Direct Embedding (HDE) - 4 qubit block
        block = int(embedding_type.split('-')[-1])
        chunk_size = 2**4  # 4 qubits
        start_index = (block - 1) * chunk_size
        end_index = min(start_index + chunk_size, len(X))
        X_chunk = X[start_index:end_index]

        if len(X_chunk) > 0:
            norm_chunk = np.linalg.norm(X_chunk)
            if norm_chunk == 0:
                raise ValueError(f"Chunk {block} has zero norm.")
            normalized_X_chunk = X_chunk / norm_chunk
            padding_chunk = [0.0] * (chunk_size - len(normalized_X_chunk))
            amplitude_data_chunk = np.concatenate((normalized_X_chunk, padding_chunk))
            qml.AmplitudeEmbedding(amplitude_data_chunk, wires=range((block - 1) * 4, block * 4), normalize=False)

    elif embedding_type.startswith('Amplitude-Hybrid2-'):
        # Hybrid Direct Embedding (HDE) - 2 qubit block
        block = int(embedding_type.split('-')[-1])
        chunk_size = 2**2  # 2 qubits
        start_index = (block - 1) * chunk_size
        end_index = min(start_index + chunk_size, len(X))
        X_chunk = X[start_index:end_index]

        if len(X_chunk) > 0:
            norm_chunk = np.linalg.norm(X_chunk)
            if norm_chunk == 0:
                raise ValueError(f"Chunk {block} has zero norm.")
            normalized_X_chunk = X_chunk / norm_chunk
            padding_chunk = [0.0] * (chunk_size - len(normalized_X_chunk))
            amplitude_data_chunk = np.concatenate((normalized_X_chunk, padding_chunk))
            qml.AmplitudeEmbedding(amplitude_data_chunk, wires=range((block - 1) * 2, block * 2), normalize=False)

    elif embedding_type.startswith('Angular-Hybrid4-'):
        # Hybrid Angle Embedding (HAE) - 4 qubit block
        block = int(embedding_type.split('-')[-1])
        start_index = (block - 1) * 4
        end_index = min(start_index + 4, len(X))
        X_chunk = X[start_index:end_index]
        for i, val in enumerate(X_chunk):
            qml.RX(val, wires=start_index + i)

    elif embedding_type.startswith('Angular-Hybrid2-'):
        # Hybrid Angle Embedding (HAE) - 2 qubit block
        block = int(embedding_type.split('-')[-1])
        start_index = (block - 1) * 2
        end_index = min(start_index + 2, len(X))
        X_chunk = X[start_index:end_index]
        for i, val in enumerate(X_chunk):
            qml.RX(val, wires=start_index + i)

    else:
        raise ValueError(f"Unknown embedding type: {embedding_type}")


# QCNN Circuit

In [33]:
import pennylane as qml

# Quantum Circuits for Convolutional layers
def conv_layer1(U, params):

    if U == 'U_TTN':
        U_TTN(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_TTN(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_TTN(params, wires=[i, i + 1])
    elif U == 'U_5':
        U_5(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_5(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_5(params, wires=[i, i + 1])
    elif U == 'U_6':
        U_6(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_6(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_6(params, wires=[i, i + 1])
    elif U == 'U_9':
        U_9(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_9(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_9(params, wires=[i, i + 1])
    elif U == 'U_13':
        U_13(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_13(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_13(params, wires=[i, i + 1])
    elif U == 'U_14':
        U_14(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_14(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_14(params, wires=[i, i + 1])
    elif U == 'U_15':
        U_15(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_15(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_15(params, wires=[i, i + 1])
    elif U == 'U_SO4':
        U_SO4(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_SO4(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_SO4(params, wires=[i, i + 1])
    elif U == 'U_SU4':
        U_SU4(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_SU4(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_SU4(params, wires=[i, i + 1])
    elif U == 'U_SU4_no_pooling':
        U_SU4(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_SU4(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_SU4(params, wires=[i, i + 1])
    elif U == 'U_SU4_1D':
        U_SU4(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_SU4(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_SU4(params, wires=[i, i + 1])
    elif U == 'U_9_1D':
        U_9(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_9(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_9(params, wires=[i, i + 1])
    else:
        print("Invalid Unitary Ansatze")
        return False

#How to visualize from the architecture??
def conv_layer2(U, params):

    if U == 'U_TTN':
        U_TTN(params, wires=[0, 6])
        U_TTN(params, wires=[0, 2])
        U_TTN(params, wires=[4, 6])
        U_TTN(params, wires=[2, 4])
    elif U == 'U_5':
        U_5(params, wires=[0, 6])
        U_5(params, wires=[0, 2])
        U_5(params, wires=[4, 6])
        U_5(params, wires=[2, 4])
    elif U == 'U_6':
        U_6(params, wires=[0, 6])
        U_6(params, wires=[0, 2])
        U_6(params, wires=[4, 6])
        U_6(params, wires=[2, 4])
    elif U == 'U_9':
        U_9(params, wires=[0, 6])
        U_9(params, wires=[0, 2])
        U_9(params, wires=[4, 6])
        U_9(params, wires=[2, 4])
    elif U == 'U_13':
        U_13(params, wires=[0, 6])
        U_13(params, wires=[0, 2])
        U_13(params, wires=[4, 6])
        U_13(params, wires=[2, 4])
    elif U == 'U_14':
        U_14(params, wires=[0, 6])
        U_14(params, wires=[0, 2])
        U_14(params, wires=[4, 6])
        U_14(params, wires=[2, 4])
    elif U == 'U_15':
        U_15(params, wires=[0, 6])
        U_15(params, wires=[0, 2])
        U_15(params, wires=[4, 6])
        U_15(params, wires=[2, 4])
    elif U == 'U_SO4':
        U_SO4(params, wires=[0, 6])
        U_SO4(params, wires=[0, 2])
        U_SO4(params, wires=[4, 6])
        U_SO4(params, wires=[2, 4])
    elif U == 'U_SU4':
        U_SU4(params, wires=[0, 6])
        U_SU4(params, wires=[0, 2])
        U_SU4(params, wires=[4, 6])
        U_SU4(params, wires=[2, 4])
    elif U == 'U_SU4_no_pooling':
        U_SU4(params, wires=[0, 6])
        U_SU4(params, wires=[0, 2])
        U_SU4(params, wires=[4, 6])
        U_SU4(params, wires=[2, 4])
    elif U == 'U_SU4_1D':
        U_SU4(params, wires=[0, 6])
        U_SU4(params, wires=[0, 2])
        U_SU4(params, wires=[4, 6])
        U_SU4(params, wires=[2, 4])
    elif U == 'U_9_1D':
        U_9(params, wires=[0, 6])
        U_9(params, wires=[0, 2])
        U_9(params, wires=[4, 6])
        U_9(params, wires=[2, 4])
    else:
        print("Invalid Unitary Ansatze")
        return False

def conv_layer3(U, params):

    if U == 'U_TTN':
        U_TTN(params, wires=[0, 4])
    elif U == 'U_5':
        U_5(params, wires=[0, 4])
    elif U == 'U_6':
        U_6(params, wires=[0, 4])
    elif U == 'U_9':
        U_9(params, wires=[0, 4])
    elif U == 'U_13':
        U_13(params, wires=[0, 4])
    elif U == 'U_14':
        U_14(params, wires=[0, 4])
    elif U == 'U_15':
        U_15(params, wires=[0, 4])
    elif U == 'U_SO4':
        U_SO4(params, wires=[0, 4])
    elif U == 'U_SU4':
        U_SU4(params, wires=[0, 4])
    elif U == 'U_SU4_no_pooling':
        U_SU4(params, wires=[0, 4])
    elif U == 'U_SU4_1D':
        U_SU4(params, wires=[0, 4])
    elif U == 'U_9_1D':
        U_9(params, wires=[0, 4])
    else:
        print("Invalid Unitary Ansatze3")
        return False

# Quantum Circuits for Pooling layers
def pooling_layer1(V, params):


    if V == 'Pooling_ansatz1':
        for i in range(0, 8, 2):
            Pooling_ansatz1(params, wires=[i + 1, i])
    elif V == 'Pooling_ansatz2':
        for i in range(0, 8, 2):
            Pooling_ansatz2(wires=[i + 1, i]) #no params
    elif V == 'Pooling_ansatz3':
        for i in range(0, 8, 2):
            Pooling_ansatz3(*params, wires=[i + 1, i]) #not limited params
    elif V=='custom_Pooling4':
            custom_Pooling4(params,wires=[0,1,2,3])
            custom_Pooling4(params,wires=[4,5,6,7])
    elif V == 'custom_Pooling1':
        custom_Pooling1(params, wires=[0, 1, 2, 3])
        custom_Pooling1(params, wires=[4, 5, 6, 7])
    elif V == 'custom_Pooling3_Mod_a':
        custom_Pooling3_Mod_a(params, wires=[0, 1, 2, 3])
        custom_Pooling3_Mod_a(params, wires=[4, 5, 6, 7])
    elif V == 'custom_Pooling3_Mod_b':
        custom_Pooling3_Mod_b(params, wires=[0, 1, 2, 3])
        custom_Pooling3_Mod_b(params, wires=[4, 5, 6, 7])
    elif V == 'custom_ansatz3_Mod_c':
        custom_ansatz3_Mod_c(params, wires=[0, 1, 2, 3])
        custom_ansatz3_Mod_c(params, wires=[4, 5, 6, 7])
    else:
        print("Invalid Pooling Ansatze")
        return False

        #how to visualize??


def pooling_layer2(V, params):
    if V == 'Pooling_ansatz1':
        Pooling_ansatz1(params, wires=[2, 0])
        Pooling_ansatz1(params, wires=[6, 4])
    elif V == 'Pooling_ansatz2':
        Pooling_ansatz2(wires=[2, 0])
        Pooling_ansatz2(wires=[6, 4])
    elif V == 'Pooling_ansatz3':
        Pooling_ansatz3(*params, wires=[2, 0])
        Pooling_ansatz3(*params, wires=[6, 4])
    elif V=='custom_Pooling4':
        custom_Pooling4(params,wires=[0,2,4,6]) #using custom_pooling1
    elif V == 'custom_Pooling1':
        custom_Pooling1(params, wires=[0, 2, 4, 6])
    elif V == 'custom_Pooling3_Mod_a':
        custom_Pooling3_Mod_a(params, wires=[0, 2, 4, 6])
    elif V == 'custom_Pooling3_Mod_b':
        custom_Pooling3_Mod_b(params, wires=[0, 2, 4, 6])
    elif V == 'custom_ansatz3_Mod_c':
        custom_ansatz3_Mod_c(params, wires=[0, 2, 4, 6])
    else:
        print("Invalid Pooling Ansatze")
        return False



def pooling_layer3(V, params):
    if V == 'Pooling_ansatz1':
        Pooling_ansatz1(params, wires=[0, 4])
    elif V == 'Pooling_ansatz2':
        Pooling_ansatz2(wires=[0, 4])
    elif V == 'Pooling_ansatz3':
        Pooling_ansatz3(*params, wires=[0, 4])
    else:
        print("Invalid Pooling Ansatze")
        return False



def QCNN_structure(U, params, U_params):
    '''U: A unitary operation that will be applied in the convolutional layers.
    params: A list of parameters to be used in the unitary operations.
    U_params: The number of parameters expected for each unitary operation.'''
    param1 = params[0:U_params[0]]
    param2 = params[U_params[0]: U_params[0]+U_params[2]]
    param3 = params[U_params[0]+U_params[2]: U_params[0]+U_params[2]+U_params[4]]
    param4 = params[U_params[0]+U_params[2]+U_params[4]: U_params[0]+U_params[2]+U_params[4]+U_params[1]]
    param5 = params[U_params[0]+U_params[2]+U_params[4]+U_params[1]: U_params[0]+U_params[2]+U_params[4]+U_params[1]+U_params[3]]
    param6 = params[U_params[0]+U_params[2]+U_params[4]+U_params[1]+U_params[3]: U_params[0]+U_params[2]+U_params[4]+U_params[1]+U_params[3]+U_params[5]]

    # Pooling Ansatz1 is used by default
    #3 conv, 3 pooling
    conv_layer1(U[0], param1)
    pooling_layer1(U[1], param4)
    conv_layer2(U[2], param2)
    pooling_layer2(U[3], param5)
    conv_layer3(U[4], param3)
    pooling_layer3(U[5], param6)


def QCNN_structure_without_pooling(U, params, U_params):
    param1 = params[0:U_params[0]]
    param2 = params[U_params[0]: U_params[0]+U_params[2]]
    param3 = params[U_params[0]+U_params[2]: U_params[0]+U_params[2]+U_params[4]]
    #No_Pooling used
    #3 conv
    conv_layer1(U[0], param1)
    conv_layer2(U[2], param2)
    conv_layer3(U[4], param3)

def QCNN_1D_circuit(U, params, U_params):#Assuming all U[0],U[2] and U[4] has QCNN_1D_circuit
    """param1 = params[0: U_params]
    param2 = params[U_params: 2*U_params]
    param3 = params[2*U_params: 3*U_params]"""
    param1 = params[0:U_params[0]]
    param2 = params[U_params[0]: U_params[0]+U_params[2]]
    param3 = params[U_params[0]+U_params[2]: U_params[0]+U_params[2]+U_params[4]]

    if U[0]=='U_SU4_1D': #just check for 1 value
        for i in range(0, 8, 2):
            U_SU4(param1, wires=[i, i + 1]) #wire 0 and 1, wire 2 and 3, wire 4 and 5, wire 6 and 7
        for i in range(1, 7, 2):
            U_SU4(param1, wires=[i, i + 1]) #wire 1 and 2, wire 3 and 4, wire 5 and 6

        U_SU4(param2, wires=[2,3]) #unitary operation U to qubits 2 and 3. Visuaalize??
        U_SU4(param2, wires=[4,5])
        U_SU4(param3, wires=[3,4])
        
    elif U[0]=='U_9_1D':
        for i in range(0, 8, 2):
            U_9(param1, wires=[i, i + 1]) #wire 0 and 1, wire 2 and 3, wire 4 and 5, wire 6 and 7
        for i in range(1, 7, 2):
            U_9(param1, wires=[i, i + 1]) #wire 1 and 2, wire 3 and 4, wire 5 and 6

        U_9(param2, wires=[2,3]) #unitary operation U to qubits 2 and 3. Visuaalize??
        U_9(param2, wires=[4,5])
        U_9(param3, wires=[3,4])

    



dev = qml.device('default.qubit', wires = 8) # A quantum device with 8 qubits is initialized using PennyLane's default qubit simulator.
@qml.qnode(dev) #The QCNN function is decorated as a QNode. This means it will execute on the specified quantum device (dev) and can compute quantum states or observables based on classical inputs.


def QCNN(X, params, U, U_params, embedding_type='Amplitude', cost_fn='cross_entropy'):


    # Data Embedding
    data_embedding(X, embedding_type=embedding_type)

    # Quantum Convolutional Neural Network
    if 'U_SU4_no_pooling' in [U[0], U[2], U[4]]: #check if we have U_SU4_no_pooling in all3 layers
        QCNN_structure_without_pooling(U, params, U_params)
    elif 'U_SU4_1D' in [U[0], U[2], U[4]]:
        QCNN_1D_circuit(U, params, U_params)
    elif 'U_9_1D' in [U[0], U[2], U[4]]:
        QCNN_1D_circuit(U, params, U_params)
    elif ('U_SU4_no_pooling' not in U) and ('U_SU4_1D' not in U) and ('U_9_1D' not in U):
        QCNN_structure(U,params,U_params)
    else:
        print("Invalid Unitary Ansatze")


    
    if cost_fn == 'mse':
        result = qml.expval(qml.PauliZ(4))
    elif cost_fn == 'cross_entropy':
        result = qml.probs(wires=4)
    return result


# Training

---

In [34]:
# Implementation of Quantum circuit training procedure
# import QCNN_circuit
# import Hierarchical_circuit
import pennylane as qml
from pennylane import numpy as np
import autograd.numpy as anp
import pennylane.numpy as qnp #new line added 11/25/24


def square_loss(labels, predictions):
    #It iterates through each pair of true label l and predicted value p, calculating the squared difference.
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    #The total loss is averaged over the number of labels.
    loss = loss / len(labels)
    return loss

def cross_entropy(labels, predictions):
    #Computes the cross-entropy loss, commonly used in binary classification tasks.
    loss = 0
    for l, p in zip(labels, predictions): #For each label-prediction pair, it calculates the cross-entropy based on whether the label is 0 or 1.
        c_entropy = l * (anp.log(p[l])) + (1 - l) * anp.log(1 - p[1 - l])
        loss = loss + c_entropy
    return -1 * loss #The total loss is returned as a negative value because cross-entropy is typically minimized.






def cost(params, X, Y, U, U_params, embedding_type, circuit, cost_fn):
    if circuit == 'QCNN':
        predictions = [QCNN(x, params, U, U_params, embedding_type, cost_fn=cost_fn) for x in X]
    
    if cost_fn == 'mse':
        loss = square_loss(Y, predictions)
    elif cost_fn == 'cross_entropy':
        loss = cross_entropy(Y, predictions)
    return loss




# Circuit training parameters
steps = 200
learning_rate = 0.01
batch_size = 25


def circuit_training(X_train, Y_train, U, U_params, embedding_type, circuit, cost_fn):
    if circuit == 'QCNN':
        total_params=0
        for i in U_params:
            if i!="":
                total_params+=int(i)
        #print(total_params)


    elif circuit == 'Hierarchical':
        #total_params = U_params * 7
        pass
    # params = qnp.random.randn(total_params, requires_grad=True)
    params = qnp.random.randn(total_params, requires_grad=True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []

    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U, U_params, embedding_type, circuit, cost_fn),
                                                     params)
        loss_history.append(cost_new)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return loss_history, params




# Benchmarking
---

In [35]:
import numpy as np

def accuracy_test(predictions, labels, cost_fn, binary = True):
        #predictions: The predicted values from the model. labels: The true labels for the data. cost_fn: A string indicating the type of cost function used in training ('mse' for mean squared error or            'cross_entropy' for cross-entropy). binary: A boolean flag indicating whether the classification is binary (default is True).
    if cost_fn == 'mse':
    
        if binary == True:
            acc = 0
            for l, p in zip(labels, predictions):
                if np.abs(l - p) < 1:
                    acc = acc + 1
            return acc / len(labels)

        else:
    
            acc = 0
            for l, p in zip(labels, predictions):
                if np.abs(l - p) < 0.5:
                    acc = acc + 1
            return acc / len(labels)

    elif cost_fn == 'cross_entropy':
        acc = 0
        for l,p in zip(labels, predictions):
            if p[0] > p[1]:
                P = 0
            else:
                P = 1
            if P == l:
                acc = acc + 1
        return acc / len(labels)


def Encoding_to_Embedding(Encoding):
    # Amplitude Embedding / Angle Embedding
    if Encoding == 'resize256':
        Embedding = 'Amplitude'
    elif Encoding == 'pca8':
        Embedding = 'Angle'
    elif Encoding == 'autoencoder8':
        Embedding = 'Angle'

    # Amplitude Hybrid Embedding
    # 4 qubit block
    elif Encoding == 'pca80-1':
        Embedding = 'Amplitude-Hybrid4-1'
    elif Encoding == 'autoencoder80-1':
        Embedding = 'Amplitude-Hybrid4-1'

    elif Encoding == 'pca80-2':
        Embedding = 'Amplitude-Hybrid4-2'
    elif Encoding == 'autoencoder80-2':
        Embedding = 'Amplitude-Hybrid4-2'

    elif Encoding == 'pca80-3':
        Embedding = 'Amplitude-Hybrid4-3'
    elif Encoding == 'autoencoder80-3':
        Embedding = 'Amplitude-Hybrid4-3'

    elif Encoding == 'pca80-4':
        Embedding = 'Amplitude-Hybrid4-4'
    elif Encoding == 'autoencoder80-4':
        Embedding = 'Amplitude-Hybrid4-4'





    elif Encoding =='pca60-1':
        Embedding = 'Amplitude-Hybrid4-1'
    elif Encoding == 'autoencoder60-1':
        Embedding = 'Amplitude-Hybrid4-1'
    
    elif Encoding =='pca60-2':
        Embedding = 'Amplitude-Hybrid4-2'
    elif Encoding == 'autoencoder60-2':
        Embedding = 'Amplitude-Hybrid4-2'
    
    elif Encoding =='pca60-3':
        Embedding = 'Amplitude-Hybrid4-3'
    elif Encoding == 'autoencoder60-3':
        Embedding = 'Amplitude-Hybrid4-3'
    elif Encoding =='pca60-4':
        Embedding = 'Amplitude-Hybrid4-4'
    elif Encoding == 'autoencoder60-4':
        Embedding = 'Amplitude-Hybrid4-4'



    elif Encoding == 'pca32-1':
        Embedding = 'Amplitude-Hybrid4-1'
    elif Encoding == 'autoencoder32-1':
        Embedding = 'Amplitude-Hybrid4-1'

    elif Encoding == 'pca32-2':
        Embedding = 'Amplitude-Hybrid4-2'
    elif Encoding == 'autoencoder32-2':
        Embedding = 'Amplitude-Hybrid4-2'

    elif Encoding == 'pca32-3':
        Embedding = 'Amplitude-Hybrid4-3'
    elif Encoding == 'autoencoder32-3':
        Embedding = 'Amplitude-Hybrid4-3'

    elif Encoding == 'pca32-4':
        Embedding = 'Amplitude-Hybrid4-4'
    elif Encoding == 'autoencoder32-4':
        Embedding = 'Amplitude-Hybrid4-4'

    # 2 qubit block
    elif Encoding == 'pca16-1':
        Embedding = 'Amplitude-Hybrid2-1'
    elif Encoding == 'autoencoder16-1':
        Embedding = 'Amplitude-Hybrid2-1'

    elif Encoding == 'pca16-2':
        Embedding = 'Amplitude-Hybrid2-2'
    elif Encoding == 'autoencoder16-2':
        Embedding = 'Amplitude-Hybrid2-2'

    elif Encoding == 'pca16-3':
        Embedding = 'Amplitude-Hybrid2-3'
    elif Encoding == 'autoencoder16-3':
        Embedding = 'Amplitude-Hybrid2-3'

    elif Encoding == 'pca16-4':
        Embedding = 'Amplitude-Hybrid2-4'
    elif Encoding == 'autoencoder16-4':
        Embedding = 'Amplitude-Hybrid2-4'

    # Angular HybridEmbedding
    # 4 qubit block
    elif Encoding == 'pca30-1':
        Embedding = 'Angular-Hybrid4-1'
    elif Encoding == 'autoencoder30-1':
        Embedding = 'Angular-Hybrid4-1'

    elif Encoding == 'pca30-2':
        Embedding = 'Angular-Hybrid4-2'
    elif Encoding == 'autoencoder30-2':
        Embedding = 'Angular-Hybrid4-2'

    elif Encoding == 'pca30-3':
        Embedding = 'Angular-Hybrid4-3'
    elif Encoding == 'autoencoder30-3':
        Embedding = 'Angular-Hybrid4-3'

    elif Encoding == 'pca30-4':
        Embedding = 'Angular-Hybrid4-4'
    elif Encoding == 'autoencoder30-4':
        Embedding = 'Angular-Hybrid4-4'

    # 2 qubit block
    elif Encoding == 'pca12-1':
        Embedding = 'Angular-Hybrid2-1'
    elif Encoding == 'autoencoder12-1':
        Embedding = 'Angular-Hybrid2-1'

    elif Encoding == 'pca12-2':
        Embedding = 'Angular-Hybrid2-2'
    elif Encoding == 'autoencoder12-2':
        Embedding = 'Angular-Hybrid2-2'

    elif Encoding == 'pca12-3':
        Embedding = 'Angular-Hybrid2-3'
    elif Encoding == 'autoencoder12-3':
        Embedding = 'Angular-Hybrid2-3'

    elif Encoding == 'pca12-4':
        Embedding = 'Angular-Hybrid2-4'
    elif Encoding == 'autoencoder12-4':
        Embedding = 'Angular-Hybrid2-4'

    # Two Gates Compact Encoding
    elif Encoding == 'pca16-compact':
        Embedding = 'Angle-compact'
    elif Encoding == 'autoencoder16-compact':
        Embedding = 'Angle-compact'
    return Embedding


def Benchmarking(dataset, classes, Unitaries, U_num_params, Encodings, circuit, cost_fn, binary=True):
    I = len(Unitaries)
    J = len(Encodings)
    for i in range(I):
        for j in range(J):
            f = open('result_mnist.txt', 'a+') # New result file for MNIST
            U = Unitaries[i]
            U_params = U_num_params[i]
            Encoding = Encodings[j]
            Embedding = Encoding_to_Embedding(Encoding)

            X_train, X_test, Y_train, Y_test = data_load_and_process(dataset, classes=classes,
                                                                     feature_reduction=Encoding, binary=binary)

            print("\n")
            print(f"Loss History for {circuit} circuits, {U} {Encoding} with {cost_fn} on {dataset}")
            loss_history, trained_params = circuit_training(X_train, Y_train, U, U_params, Embedding, circuit, cost_fn)

            if circuit == 'QCNN':
                predictions = [QCNN(x, trained_params, U, U_params, Embedding, cost_fn) for x in X_test]

            accuracy = accuracy_test(predictions, Y_test, cost_fn, binary)
            print(f"Accuracy for {U} {Encoding} on {dataset}: {accuracy}")

            f.write(f"Loss History for {circuit} circuits, {U} {Encoding} with {cost_fn} on {dataset}\n")
            f.write(str(loss_history))
            f.write(f"\nAccuracy for {U} {Encoding} on {dataset}: {accuracy}\n\n")
            f.close()





def Data_norm(dataset, classes, Encodings, binary=True):
    J = len(Encodings) #J: The number of different encoding methods provided in the Encodings list.
    Num_data = 10000 #Num_data: A constant set to 10,000, representing the number of samples to be randomly selected for normalization calculations.

    f = open('Result/data_norm.txt', '+a')

    for j in range(J):
        Encoding = Encodings[j]

        X_train, X_test, Y_train, Y_test = data_load_and_process(dataset, classes=classes,
                                                                          feature_reduction=Encoding, binary=binary)

        #norm calculation for pca32-3 and autoencoder32-3
        if Encoding == 'pca32-3' or Encoding == 'autoencoder32-3':
            norms_X1 = []
            norms_X2 = []
            for i in range(Num_data):
                index = np.random.randint(0, len(X_train))
                X = X_train[index]

                X1 = X[:2 ** 4]
                X2 = X[2 ** 4:2 ** 5]
                norm_X1, norm_X2 = np.linalg.norm(X1), np.linalg.norm(X2)
                norms_X1.append(norm_X1)
                norms_X2.append(norm_X2)

            norms_X1, norms_X2 = np.array(norms_X1), np.array(norms_X2)
            mean_X1, stdev_X1 = np.mean(norms_X1), np.std(norms_X1)
            mean_X2, stdev_X2 = np.mean(norms_X2), np.std(norms_X2)

            if Encoding == 'pca32-3':
                f.write("PCA32 Encoding\n")
            elif Encoding == 'autoencoder32-3':
                f.write("autoencoder32 Encoding\n")
            f.write("mean of X1: " + str(mean_X1) + " standard deviation of X1: " + str(stdev_X1))
            f.write("\n")
            f.write("mean of X2: " + str(mean_X2) + " standard deviation of X2: " + str(stdev_X2))
            f.write("\n")

        #if pca16 or autoencoder16 is used

        elif Encoding == 'pca16' or Encoding == 'autoencoder16':

            norms_X1 = []
            norms_X2 = []
            norms_X3 = []
            norms_X4 = []
            for i in range(Num_data):
                index = np.random.randint(0, len(X_train))
                X = X_train[index]

                X1 = X[:4]
                X2 = X[4:8]
                X3 = X[8:12]
                X4 = X[12:16]
                norm_X1, norm_X2, norm_X3, norm_X4 = np.linalg.norm(X1), np.linalg.norm(X2), np.linalg.norm(
                    X3), np.linalg.norm(X4)

                norms_X1.append(norm_X1)
                norms_X2.append(norm_X2)
                norms_X3.append(norm_X3)
                norms_X4.append(norm_X4)

            norms_X1, norms_X2, norms_X3, norms_X4 = np.array(norms_X1), np.array(norms_X2), np.array(norms_X3), np.array(norms_X4)

            mean_X1, stdev_X1 = np.mean(norms_X1), np.std(norms_X1)
            mean_X2, stdev_X2 = np.mean(norms_X2), np.std(norms_X2)
            mean_X3, stdev_X3 = np.mean(norms_X3), np.std(norms_X3)
            mean_X4, stdev_X4 = np.mean(norms_X4), np.std(norms_X4)

            if Encoding == 'pca16':
                f.write("PCA16 Encoding\n")
            elif Encoding == 'autoencoder16':
                f.write("autoencoder16 Encoding\n")
            f.write("mean of X1: " + str(mean_X1) + " standard deviation of X1: " + str(stdev_X1))
            f.write("\n")
            f.write("mean of X2: " + str(mean_X2) + " standard deviation of X2: " + str(stdev_X2))
            f.write("\n")
            f.write("mean of X3: " + str(mean_X3) + " standard deviation of X3: " + str(stdev_X3))
            f.write("\n")
            f.write("mean of X4: " + str(mean_X4) + " standard deviation of X4: " + str(stdev_X4))
            f.write("\n")

    f.close()


# Result

---

In [36]:
Conv_Pooling_Layer_Ansatzs = [['U_SU4',"custom_Pooling1",'U_SU4','custom_Pooling4','U_SU4','Pooling_ansatz1']]
Conv_Pooling_Layer_Params = [[15,10,15,14,15,2]]
Encodings = ['resize256', 'pca8', 'autoencoder8'] # Example encodings
dataset = 'mnist'
classes = [0, 1]
binary = False
cost_fn = 'cross_entropy'

Benchmarking(dataset, classes, Conv_Pooling_Layer_Ansatzs, Conv_Pooling_Layer_Params, Encodings, circuit='QCNN', cost_fn=cost_fn, binary=binary)


Loaded X_train shape: (60000, 28, 28, 1), Y_train shape: (60000,)
Loaded X_test shape: (10000, 28, 28, 1), Y_test shape: (10000,)


Loss History for QCNN circuits, ['U_SU4', 'custom_Pooling1', 'U_SU4', 'custom_Pooling4', 'U_SU4', 'Pooling_ansatz1'] resize256 with cross_entropy on mnist


c:\Users\shahr\anaconda3\envs\audio_qcnn\Lib\site-packages\autograd\numpy\numpy_vjps.py:698: ComplexWarning: Casting complex values to real discards the imaginary part
  onp.add.at(A, idx, x)


iteration:  0  cost:  18.60062484754571
iteration:  10  cost:  14.177053812089383
iteration:  20  cost:  11.294621030517568
iteration:  30  cost:  11.773032528390786
iteration:  40  cost:  10.743943750322186
iteration:  50  cost:  9.406224956323598
iteration:  60  cost:  9.70471012916364
iteration:  70  cost:  8.07365115099519
iteration:  80  cost:  9.293423446161768
iteration:  90  cost:  8.936476654693967
iteration:  100  cost:  8.726114091481861
iteration:  110  cost:  8.478841593663232
iteration:  120  cost:  9.47719376345326
iteration:  130  cost:  7.611530852358742
iteration:  140  cost:  7.597693612025682
iteration:  150  cost:  6.021387190374452
iteration:  160  cost:  6.868364086716073
iteration:  170  cost:  5.817898721344762
iteration:  180  cost:  7.5266157179329065
iteration:  190  cost:  7.032286095663909
Accuracy for ['U_SU4', 'custom_Pooling1', 'U_SU4', 'custom_Pooling4', 'U_SU4', 'Pooling_ansatz1'] resize256 on mnist: 0.983451536643026
Loaded X_train shape: (60000, 28,

c:\Users\shahr\anaconda3\envs\audio_qcnn\Lib\site-packages\autograd\numpy\numpy_vjps.py:698: ComplexWarning: Casting complex values to real discards the imaginary part
  onp.add.at(A, idx, x)


iteration:  0  cost:  17.31434531008349
iteration:  10  cost:  11.433238205750566
iteration:  20  cost:  10.660392061954143
iteration:  30  cost:  10.865641201324236
iteration:  40  cost:  10.294000447833175
iteration:  50  cost:  12.418696351704481
iteration:  60  cost:  5.826112987152229
iteration:  70  cost:  8.920774229047208
iteration:  80  cost:  4.548406444269806
iteration:  90  cost:  3.5729420852304474
iteration:  100  cost:  9.562437358303644
iteration:  110  cost:  5.404019596097008
iteration:  120  cost:  4.269662669078811
iteration:  130  cost:  5.473831038767254
iteration:  140  cost:  3.5100788170854864
iteration:  150  cost:  5.732372188802917
iteration:  160  cost:  6.543980067916802
iteration:  170  cost:  4.9429418233949685
iteration:  180  cost:  4.398967240598519
iteration:  190  cost:  4.8673676045680745
Accuracy for ['U_SU4', 'custom_Pooling1', 'U_SU4', 'custom_Pooling4', 'U_SU4', 'Pooling_ansatz1'] pca8 on mnist: 0.9791962174940898
Loaded X_train shape: (60000, 